# Initialization

Test notebook for the C-MAPPS benchmark. Test the Tunable Model class architecture

First we import the necessary packages and create the global variables.

In [1]:
import math
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import custom_scores
import CMAPSAuxFunctions
from tunableModel import TunableModel

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers
from numpy.random import seed
seed(1)

import tensorflow as tf

from scipy.optimize import differential_evolution

from numpy.random import seed
seed(1)

Using TensorFlow backend.


# Create models

Create different types of models to be tested with the Tunable Model class

In [2]:
#Clear the previous tensorflow graph
K.clear_session()

FilterN = 10
FilterL = 10

lambda_regularization = 0.20

def RULmodel_SN_1(input_shape):
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(30, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dropout(0.4))
    model.add(Dense(10, activation='relu', kernel_initializer='glorot_normal', name='fc2'))
    model.add(Dropout(0.1))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model

def RULmodel_SN_2(input_shape):
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(lambda_regularization), name='fc1'))
    #model.add(Dense(30, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
    #                kernel_regularizer=regularizers.l2(0.1), name='fc1'))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model


def RULModel_CNN_1(input_shape):
    
    #Create a sequential model
    model = Sequential()
    
    print(input_shape)
    
    #Add the layers for the model
    model.add(Conv2D(1, (FilterN, FilterL), input_shape=input_shape, padding='same', kernel_initializer='glorot_normal', activation='relu', name='cl1'))
    model.add(Dropout(0.6))
    model.add(MaxPooling2D(pool_size=5, name='pl1'))
    model.add(Flatten())
    model.add(Dense(20, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model


def RULCNNModel(TW, FeatureN):
    
    input_layer = Input(shape=(TW, FeatureN))
    y = Reshape((TW, FeatureN, 1), input_shape=(TW, FeatureN, ),name = 'Reshape')(input_layer)

    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C1')(y)
    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C2')(y)
    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C3')(y)
    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C4')(y)
    #y = Convolution2D(FilterN, FilterL, 1, border_mode='same', init='glorot_normal', activation='tanh', name='C5')(y)
    #y = Convolution2D(FilterN, FilterL, 1, border_mode='same', init='glorot_normal', activation='tanh', name='C6')(y)
    
    y = Conv2D(1, 3, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='Clast')(y)  
    
    y = Reshape((TW,14))(y)
    y = Flatten()(y)
    y = Dropout(0.5)(y)
    
    #y = Dense(100, activation='tanh', init='glorot_normal', activity_regularizer=keras.regularizers.l2(0.01),)(y)
    y = Dense(100,activation='tanh', kernel_initializer='glorot_normal', name='fc')(y)
    y = Dense(1)(y)
    
    model = Model(inputs = input_layer, outputs = y, name='RUL_CNN_Model')
    
    return model

# Load the data

In [3]:
#Shared parameters for the models
optimizer = Adam(lr=0, beta_1=0.5)
lossFunction = "mean_squared_error"
metrics = ["mse"]

#Selected as per CNN paper
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']

selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
#selected_indices = np.array([7, 8, 9, 12, 16, 17, 20])
#selected_indices = np.array(list(range(1,22)))

selected_features = list(features[i] for i in selected_indices-1)

print(selected_features)

windowSize = 30
stride = 1
constRUL = 128


dataFolder = '../../CMAPSSData'

#Create and compile the models
nFeatures = len(selected_features)
shapeSN = nFeatures*windowSize
modelRULSN = RULmodel_SN_2(shapeSN)
modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

tModel = TunableModel('ModelRUL_SN_1', modelRULSN, selected_features, dataFolder, 'keras', window_size=windowSize,
                      scaler = min_max_scaler, window_stride=stride, constRul = constRUL)
tModel.changeDataset(1)
tModel.loadData(verbose=1, rectify_labels = False)
tModel.printData()

['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 'BPR', 'htBleed', 'W31', 'W32']
Loading data for dataset 1 with window_size of 30, stride of 1 and constRUL of 128. Cros-Validation ratio 0
Data loaded for dataset 1
Printing shapes

Training data (X, y)
(17731, 420)
(17731,)
Testing data (X, y)
(100, 420)
(100,)
Printing first 5 elements

Training data (X, y)
[[-0.58075601 -0.0455243  -0.27982732 ... -0.81818182  0.43307087
   0.4679733 ]
 [-0.35395189  0.0629156  -0.18014129 ... -0.45454545  0.25984252
   0.25294702]
 [-0.21649485 -0.13299233 -0.13854003 ... -0.45454545  0.38582677
   0.72049425]
 [-0.21649485 -0.39897698 -0.2299843  ... -0.45454545  0.08661417
   0.29640676]
 [-0.20274914 -0.39590793 -0.05926217 ... -0.45454545  0.05511811
   0.17880983]]
[128. 128. 128. 128. 128.]
Testing data (X, y)
[[-0.65635739 -0.10946292 -0.48312402 ... -0.27272727  0.05511811
   0.30947309]
 [ 0.03780069 -0.07365729 -0.27629513 ... -0.63636364  0.05511811
   0.04416986]
 [ 

In [4]:
#tModel.printData(printTop=False)
#tModel.windowSize = 20
#tModel.changeDataset(2)
#tModel.loadData(verbose=1, crossValRatio=0.1)
#tModel.printData(printTop=False)
tModel.getModelDescription()
print(tModel.batch_size)
#ruldcnn = RULCNNModel(tModel.windowSize, len(selected_features))
#ruldcnn.summary()
#tModel.trimmedRUL_train
#display(tModel.df_train)

Description for model: ModelRUL_SN_1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fc1 (Dense)                  (None, 20)                8420      
_________________________________________________________________
out (Dense)                  (None, 1)                 21        
Total params: 8,441
Trainable params: 8,441
Non-trainable params: 0
_________________________________________________________________
512


In [5]:
def print_model_weights(model):

    for layer in model.layers:
        weights = layer.get_weights() # list of numpy arrays
        print(weights)

# Train the model and test some Tunable Model functionalities

In [10]:
lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
#lrate = None

#print(tModel.currentModelName)
#tModel.getAllModelsDescription()
#tModel.getModelDescription('RULModel_CNN_1')
#tModel.getModelDescription(tModel.currentModelName)
#tModel.getModelDescription()
#tModel.setCurrentModel('RULmodel_SN_1')
#tModel.getModelDescription()
#Create and compile the models
nFeatures = len(selected_features)
shapeSN = nFeatures*windowSize
modelRULSN = None
modelRULSN = RULmodel_SN_2(shapeSN)
modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
tModel.changeModel('ModelRUL_SN_1', modelRULSN, 'keras')

print("Printing model weights")
print_model_weights(tModel.model)

tModel.epochs = 10
tModel.trainModel(learningRateScheduler=lrate, verbose=1)
print_model_weights(tModel.model)
#K.clear_session()
#tf.reset_default_graph()

Printing model weights
[array([[ 0.01325954,  0.03105452,  0.01612089, ..., -0.02751129,
         0.00941739,  0.01329393],
       [-0.09007745,  0.03733502,  0.0183803 , ...,  0.04441739,
        -0.00659874, -0.03448449],
       [-0.03554275,  0.04322699,  0.11718896, ...,  0.1001227 ,
         0.05780441,  0.03238355],
       ...,
       [-0.01029107, -0.07319549, -0.01854422, ...,  0.01316447,
         0.0100575 ,  0.05907865],
       [-0.03283691, -0.00276536, -0.11672528, ..., -0.0707932 ,
         0.04540976, -0.02643326],
       [-0.0166082 ,  0.02198435,  0.05619126, ..., -0.03184278,
         0.06302007, -0.03821027]], dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)]
[array([[-4.1883007e-01],
       [ 2.0567465e-01],
       [-7.0012271e-02],
       [ 4.0628332e-01],
       [-3.3603531e-01],
       [-5.2185851e-01],
       [-1.2382868e-01],
       [-5.1254362e-02],
       [ 2.6712358e-02],
       [ 1

In [12]:
tModel.evaluateModel(["rhs"], round=2, setLimits=[])
print("scores")

#print(tModel.y_pred)

cScores = tModel.scores
rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.trainTime

i = 1
for prediction in tModel.y_pred:
    print('Engine {}, RUL {}, Rounded RUL {}, Real RUL {}'.format(i, prediction, 
                                                                    tModel.y_pred_rectified[i-1], 
                                                                    tModel.y_test[i-1]))
    i = i+1

print(cScores)
print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

100/100 [==============================] - 0s 359us/step
scores
Engine 1, RUL [115.31297], Rounded RUL 115.0, Real RUL 112.0
Engine 2, RUL [116.61171], Rounded RUL 116.0, Real RUL 98.0
Engine 3, RUL [60.587223], Rounded RUL 60.0, Real RUL 69.0
Engine 4, RUL [81.65829], Rounded RUL 81.0, Real RUL 82.0
Engine 5, RUL [96.753685], Rounded RUL 96.0, Real RUL 91.0
Engine 6, RUL [101.09155], Rounded RUL 101.0, Real RUL 93.0
Engine 7, RUL [106.26153], Rounded RUL 106.0, Real RUL 91.0
Engine 8, RUL [88.65305], Rounded RUL 88.0, Real RUL 95.0
Engine 9, RUL [115.34675], Rounded RUL 115.0, Real RUL 111.0
Engine 10, RUL [88.02823], Rounded RUL 88.0, Real RUL 96.0
Engine 11, RUL [90.10341], Rounded RUL 90.0, Real RUL 97.0
Engine 12, RUL [101.78441], Rounded RUL 101.0, Real RUL 124.0
Engine 13, RUL [82.6121], Rounded RUL 82.0, Real RUL 95.0
Engine 14, RUL [108.17689], Rounded RUL 108.0, Real RUL 107.0
Engine 15, RUL [106.09766], Rounded RUL 106.0, Real RUL 83.0
Engine 16, RUL [107.45257], Rounded RUL

In [4]:
def plotRUL(cycles, rulArray, engineUnit):
    
    plt.clf()
    plt.plot(cycles, rulArray, 'bo-', label='RUL')
    plt.xlabel("Time (Cycle)")
    plt.ylabel("RUL")
    plt.title("Test Engine Unit #{}".format(engineUnit))
    
    plt.show()
    
def plotRUL_Engine(tModel, engineUnit, selectedFeatures, time_window, constRUL, stride, y_test):

    df = tModel.df_test
    df_engine = df[df['Unit Number'] == engineUnit]
    
    '''
    X_test2, _, _, _ = CMAPSAuxFunctions.create_windowed_data(df_engine, selected_features, 'test', time_window=time_window, 
                                                     constRUL=constRUL, stride=stride)
    
    X_test2 = tModel.dataScaler.transform(X_test2)
    nnPred = tModel.model.predict(X_test2)
    '''
    
    maxCycle = X_test2.shape[0]
    faultCycle = tModel.y_test[engineUnit-1]
    cycles = np.arange(maxCycle)
    rulArray = np.arange(faultCycle, maxCycle+faultCycle)
    rulArray[rulArray > constRUL] = constRUL
    rulArray = np.flipud(rulArray)
    
    plotRUL(cycles, rulArray, engineUnit)
    #plt.plot(cycles, nnPred, 'go-', label='NN Pred')
    plt.legend()

In [5]:
plotRUL_Engine(tModel, engineUnit, selectedFeatures, time_window, stride, constRUL)

NameError: name 'engineUnit' is not defined